## Analyze A/B Test Results

You may either submit your notebook through the workspace here, or you may work from your local machine and submit through the next page.  Either way assure that your code passes the project [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).  **Please save regularly.**

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)
- [Conclusions](#conclusions)

<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

/opt/conda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [2]:
df = pd.read_csv('ab_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


b. Use the cell below to find the number of rows in the dataset.

In [3]:
df.shape

(294478, 5)

c. The number of unique users in the dataset.

In [4]:
unique_users = df.user_id.nunique()
unique_users

290584

d. The proportion of users converted.

In [5]:
prop_converted_users = df.query('converted == 1').user_id.nunique()/ unique_users
prop_converted_users

0.12104245244060237

e. The number of times the `new_page` and `treatment` don't match.

In [6]:
df.query('(group == "treatment" and landing_page != "new_page") or (group != "treatment" and landing_page == "new_page") ').count()

user_id         3893
timestamp       3893
group           3893
landing_page    3893
converted       3893
dtype: int64

f. Do any of the rows have missing values?

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
user_id         294478 non-null int64
timestamp       294478 non-null object
group           294478 non-null object
landing_page    294478 non-null object
converted       294478 non-null int64
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


`2.` For the rows where **treatment** does not match with **new_page** or **control** does not match with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to figure out how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [8]:
df2 = df.copy()
df2.sample(10)

,user_id,timestamp,group,landing_page,converted
56260,679354,2017-01-19 20:27:09.540303,treatment,new_page,0
21639,792591,2017-01-23 19:43:13.523961,treatment,new_page,0
73671,786633,2017-01-24 10:43:37.105184,treatment,new_page,0
199271,936842,2017-01-08 17:02:37.417670,treatment,new_page,0
41703,839604,2017-01-09 08:30:56.894297,control,old_page,1
234324,807408,2017-01-15 21:37:02.195283,control,old_page,0
63187,936867,2017-01-22 03:33:56.931492,treatment,new_page,0
211054,678788,2017-01-20 06:47:04.343780,treatment,new_page,1
7931,776710,2017-01-19 01:13:50.532410,control,old_page,1
71401,829264,2017-01-07 06:15:31.282411,treatment,new_page,0


In [9]:
indeces = df2.query('(group == "treatment" and landing_page != "new_page") or (group != "treatment" and landing_page == "new_page") ').index
df2.drop(indeces
         , inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290585 entries, 0 to 294477
Data columns (total 5 columns):
user_id         290585 non-null int64
timestamp       290585 non-null object
group           290585 non-null object
landing_page    290585 non-null object
converted       290585 non-null int64
dtypes: int64(2), object(3)
memory usage: 13.3+ MB


In [10]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]
df2.shape[0], df.shape[0]

(290585, 294478)

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [11]:
df2.user_id.nunique()

290584

b. There is one **user_id** repeated in **df2**.  What is it?

In [12]:
df2[df2.duplicated(subset=['user_id'])==True].user_id

2893    773192
Name: user_id, dtype: int64

c. What is the row information for the repeat **user_id**? 

In [13]:
df2.query('user_id == 773192')

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [14]:
#dropping the second row for the same user
df2.drop(df2.query('user_id == 773192').index[1], inplace=True)

In [15]:
#check that we correctly removed the second row
df2.query('user_id == 773192')

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0


In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290584 entries, 0 to 294477
Data columns (total 5 columns):
user_id         290584 non-null int64
timestamp       290584 non-null object
group           290584 non-null object
landing_page    290584 non-null object
converted       290584 non-null int64
dtypes: int64(2), object(3)
memory usage: 13.3+ MB


d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

`4.` Use **df2** in the cells below to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [17]:
converting_prop = df2.query('converted == 1').user_id.count()/df2.shape[0]
converting_prop

0.11959708724499628

b. Given that an individual was in the `control` group, what is the probability they converted?

In [18]:
control_df = df2.query('group == "control"')
treatment_df = df2.query('group == "treatment"')

converting_prop_control = control_df.query('converted == 1').user_id.count()/control_df.shape[0]
converting_prop_control

0.1203863045004612

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [19]:
converting_prop_treatment = treatment_df.query('converted == 1').user_id.count()/treatment_df.shape[0]
converting_prop_treatment

0.11880806551510564

d. What is the probability that an individual received the new page?

In [20]:
receiving_new_page_prop = df2.query('landing_page == "new_page"').user_id.count()/df2.shape[0]
receiving_new_page_prop

0.50006194422266881

e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.

**Although it is statistically significant to say that the new page leads to slightly more converting users, practically is not strong evidence to say that since the difference between control and treatment datasets in terms of converting users proportion(less than 0.08% diff) which is very small for such big dataset.**

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

$$ H_{0}: P_{new} - P_{old}  \leq 0 $$
$$ H_{1}: P_{new} - P_{old} > 0 $$

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

a. What is the **conversion rate** for $p_{new}$ under the null? 

In [21]:
newPage_df= df2.query('landing_page == "new_page"')
p_new = df2.converted.mean()
p_new

0.11959708724499628

b. What is the **conversion rate** for $p_{old}$ under the null? <br><br>

In [22]:
oldPage_df= df2.query('landing_page == "old_page"')
p_old = df2.converted.mean()
p_old

0.11959708724499628

In [23]:
p_new - p_old

0.0

c. What is $n_{new}$, the number of individuals in the treatment group?

In [24]:
n_new =newPage_df.user_id.count()
n_new

145310

d. What is $n_{old}$, the number of individuals in the control group?

In [25]:
n_old =oldPage_df.user_id.count()
n_old

145274

e. Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [26]:
new_page_converted = np.random.choice([0,1],n_new, p=(1-p_new,p_new))

f. Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [27]:
old_page_converted = np.random.choice([0,1],n_old, p=(1-p_old,p_old))

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [28]:
new_page_converted.mean() - old_page_converted.mean()

-0.0020390322735635291

h. Create 10,000 $p_{new}$ - $p_{old}$ values using the same simulation process you used in parts (a) through (g) above. Store all 10,000 values in a NumPy array called **p_diffs**.

In [ ]:
p_diffs = []

for n in range(10000):
    new_page_converted = np.random.choice([0,1],n_new, p=(1-p_new,p_new))
    old_page_converted = np.random.choice([0,1],n_old, p=(1-p_old,p_old))
    p_diffs.append(new_page_converted.mean() - old_page_converted.mean())
    print(str(n) + ' / 10000      ', end='\r') #just to check where the simulation is

print(len(p_diffs))
    


i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
plt.hist(p_diffs);

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
null_vals = np.random.normal(0, np.array(p_diffs).std(), len(p_diffs))
plt.hist(null_vals);

In [ ]:
# Actual observed differences in conversion, infered from ab_data.csv
obs_p_diff = df2[df2['group'] == 'treatment']['converted'].mean() -  df2[df2['group'] == 'control']['converted'].mean()
print(obs_p_diff)

In [ ]:
(null_vals > obs_p_diff).mean()

k. Please explain using the vocabulary you've learned in this course what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

**This value is called the p_value, it means the alternative hypothesis(new_page is better than the old_page) is significant when the null hypothesis is true(the is no difference).**

> Here our p_value is much larger than alpha(0.05) which indicates that we can't reject the null hypothesis, which means that the old page is better than the new page. 

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
# import statsmodels.api as sm

convert_old = df2.query('landing_page == "old_page" and converted == 1').user_id.count()
convert_new = df2.query('landing_page == "new_page" and converted == 1').user_id.count()
n_old = df2.query('landing_page == "old_page"').user_id.count()
n_new =  df2.query('landing_page == "new_page"').user_id.count()

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](https://docs.w3cub.com/statsmodels/generated/statsmodels.stats.proportion.proportions_ztest/) is a helpful link on using the built in.

In [ ]:
z_score, p_value = proportions_ztest([convert_old, convert_new], [n_old, n_new], alternative='smaller' )
z_score, p_value


n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

**z-score: is the standard binomial random variable that shows the diff between conversion rates for old and new pages
<br>
<br>

p-value: is to determine the probability associated with z-score>z-obs, to test if the new page is better than the old page(our alternative hypothesis), since the p-value is much larger than alpha(0.05) this means we can't reject the null hypothesis meaning that the old page is better**
<hr>
**Yes, both p_values are equal.**

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

**Logistic Regression**

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives. However, you first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
df2['intercept'] = 1

df2[['_','ab_page']] = pd.get_dummies(df2.group)
df2.drop(columns={'_'}, inplace = True)
df2.head()

c. Use **statsmodels** to instantiate your regression model on the two columns you created in part b., then fit the model using the two columns you created in part **b.** to predict whether or not an individual converts. 

In [ ]:
model1 = sm.Logit(df2.converted, df2[['intercept', 'ab_page']])
result1 = model1.fit()

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
result1.summary2()

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in **Part II**?

**P_value = 0.1897 since our hypotheses differ from part II and since we are using a two-sided t-test in partIII rather one-sided t-test in part II**

> **For our logistic model** <br>
**Null hypothesis**: ab_page isn't mainly responsible for conversion <br>
**Alternative hypothesis**: ab_page is mainly responsible for conversion<br>
$$H0: P_{new}−P_{old}=0$$
$$H1: P_{new}−P_{old}\neq0$$
<hr>

> **For part II**<br>
**Null hypothesis**: the proportion of converted users using new_page is less than or equal the proportion of converted users using old_page<br>
**Alternative hypothesis**: the proportion of converted users using new_page is larger than the proportion of converted users using old_page<br>
$$H0: P_{new}−P_{old}\leq0$$
$$H1: P_{new}−P_{old}>0$$

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

#### Advantage of adding more factors:
- Build a solid model that takes most factors that affect whether or not an individual converts.
- Adding interaction or/and higher-order terms is great when some factor is related to another.

#### Disadvantage of adding more factors:
- Sometimes when adding numeric factors they can cause multicollinearity since they are correlated to each other.
- Adding higher-order factors make them dependent on other factors thus, making interpretation of the model harder and nonlinear.
- Another problem that could arise by considering other additional factors may be Simpson's Paradox,

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. You will need to read in the **countries.csv** dataset and merge together your datasets on the appropriate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [ ]:
countries_df = pd.read_csv('countries.csv')
countries_df.info()
countries_df.head()

In [ ]:
df2 = df2.join(countries_df.set_index('user_id'), on='user_id')

In [ ]:
df2[['CA', 'UK','US']] = pd.get_dummies(df2.country)
df2.sample(5)

In [ ]:
#taking US as the baseline for UK and CA 
model2 = sm.Logit(df2.converted, df2[['intercept', 'ab_page', 'CA','UK']])
result2 = model2.fit()
result2.summary2()

**p_values of the UK and CA show that they are not statistically significant so countries have no effect on the conversion of an individual .**

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
df2['ab_page_US'] = df2['ab_page'] * df2['US']
df2['ab_page_UK'] = df2['ab_page'] * df2['UK']

model3 = sm.Logit(df2['converted'], df2[['intercept','ab_page','US','UK','ab_page_US','ab_page_UK']])
results3 = model3.fit()
results3.summary2()

**adding the interaction terms to our model couldn't also help us to see which explanatory variables are associated with the conversion rate since the p_value for `ab_page_US` and `ab_page_UK` is much higher than our alpha, which states that they are not statistically significant.**

<a id='conclusions'></a>
## Conclusions
Just to recap we need to remember that our main goal of this A/B testing is to determine whether the new page is better than the old page or not based on the conversion rate so that we decide if the new page worth implementing rather than stick to the old page. 

To wrap up our A/B testing we should interpret and conclude our results and findings throughout the entire notebook, so found the following:
<br>

### **For Part I**<br>

- 290584 were unique users out of 294478 users in the dataset.
- 3894 weren't unique users (were dropped later), as follows:
- -3893 users didn't match between the new_page and treatment.
- - 1 user with user_id of '773192' had two duplicated rows.
- Conversion rate regardless of the page was 11.96%, but the conversion rate was 12.04% and 11.88% for control and treatment groups, respectively.
> Although it is statistically significant to say that the new page leads to slightly more converting users, practically is not strong evidence to say that since the difference between control and treatment datasets in terms of converted users proportion(less than 0.08% diff) which is very small for such big dataset.

<hr>

### **For Part II**<br>
We chose our hypotheses to be the following:<br>
**Null hypothesis**: ab_page isn't mainly responsible for conversion <br>
**Alternative hypothesis**: ab_page is mainly responsible for conversion<br>
$$H0: P_{new}−P_{old}=0$$
$$H1: P_{new}−P_{old}\neq0$$
<br>
- Since we claim an equal converting rate in ab_data.csv regardless of the page, we found that the conversion rate for both the old page and the new page was 11.95% each meaning that there is no difference in the conversion rate.
- we had 145310 users in the treatment group and 145274 users in the control group.
- We Simulate  $𝑛_{𝑛𝑒𝑤}$  transactions with a conversion rate of  $𝑝_{𝑛𝑒𝑤}$ and $𝑛_{old}$  transactions with a conversion rate of  $𝑝_{old}$ under the null and generated 10,000 `p_diffs` and after plotting it we found that it has a binomial distribution shape which we expected as this dataset obey central limit theorem.
- `obs_p_diff` was approximately -0.0016, meaning that there is  about 90.5%(p-value * 100 ) of p_diffs were larger than our `obs_p_diff`.
- We computed the p-value again, this time using the upper side z-test and we got the same p-value

> Here our p_value is much larger than alpha(0.05 or 95% confidence level) which indicates that is no statistical evidence to reject the null hypothesis, which means that the old page is better than the new page. 

<hr>

### **For Part III**<br>
We chose our hypotheses to be the following:<br>
**Null hypothesis**: the proportion of converted users using new_page is less than or equal the proportion of converted users using old_page<br>
**Alternative hypothesis**: the proportion of converted users using new_page is larger than the proportion of converted users using old_page<br>
$$H0: P_{new}−P_{old}\leq0$$
$$H1: P_{new}−P_{old}>0$$
<br>
Since we are curious to predict if a user is going to convert or not we will be using Logistic Regression.<br> 

- Building our first model using `ab_page` as our exploratory variable, we found that the p-value for 'ab_page' was 0.1899 so we can't use ab_page to predict if the user will convert or not, and we can find that the p-value here (associated with two-sided t-test) differs from the one we found in part II (associated with upper-side t-test).
<br>
- Building our second model using `ab_page`, `CA`, and `UK` and taking `US` as our baseline and p_values of UK and CA show that they are not statistically significant so countries have no effect on the conversion of an individual.
<br>
- Building our second model using `ab_page`, `CA`, `UK`, `ab_page_US`, `ab_page_UK` and taking `US` as our baseline.
Adding the interaction terms to our model couldn't also help us to see which explanatory variables are associated with the conversion rate since the p_value for `ab_page_US` and `ab_page_UK` is much higher than our alpha, which states that they are not statistically significant <br>
> Overall all models showed that all explanatory variables we used were not statistically significant, which lead us to not reject the null hypothesis since the old page is better than the new page.

<hr>
<hr>

- So based on our findings and results in it's safe to say that a new page won't do us any favor in terms of increase users' conversion rate in fact developing the new page can take effort, time, and money with no practical outcome from it.

- Finally, improving our regression model would need more numerical attributes about our users to help us understand their behavior better to best estimate which factors are most likely leading users to convert.

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Analyze_ab_test_results_notebook.ipynb'])